4.AutoGluon

In [1]:
import pandas as pd
# load the dataset
df = pd.read_csv('household_power_consumption.txt', delimiter=';', low_memory=False)
# grab time series of electricity usage
df['date_time'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])
df['electricity_usage'] = pd.to_numeric(df['Global_active_power'], errors='coerce')
df = df[['date_time', 'electricity_usage']].copy()
# set the index as and sort by date and time
df = df.set_index('date_time').sort_index()
# resample to hourly frequency
df_hour = df.resample('1H').mean()
# create shifted time series
df_hour['electricity_usage'] = df_hour['electricity_usage'].fillna(method='ffill')
df_hour['electricity_usage_1hr_lag'] = df_hour['electricity_usage'].shift()
df_hour['electricity_usage_2hr_lag'] = df_hour['electricity_usage'].shift(2)
df_hour['electricity_usage_3hr_lag'] = df_hour['electricity_usage'].shift(3)
df_hour['electricity_usage_4hr_lag'] = df_hour['electricity_usage'].shift(4)
df_hour['electricity_usage_5hr_lag'] = df_hour['electricity_usage'].shift(5)
df_hour['electricity_usage_6hr_lag'] = df_hour['electricity_usage'].shift(6)
df_hour['electricity_usage_7hr_lag'] = df_hour['electricity_usage'].shift(7)
df_hour['electricity_usage_8hr_lag'] = df_hour['electricity_usage'].shift(8)
# create month/seasonality as a feature
df_hour['month'] = df_hour.index.month
# drop missing data
df_hour = df_hour.dropna()
# split train test sets
sample_size = df_hour.shape[0] - 200
df_train = df_hour.iloc[:sample_size].copy()
df_test = df_hour.iloc[sample_size:].copy()

C:\Users\jv028u\AppData\Local\Temp\ipykernel_2960\2681619798.py:5: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['date_time'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])
C:\Users\jv028u\AppData\Local\Temp\ipykernel_2960\2681619798.py:13: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_hour['electricity_usage'] = df_hour['electricity_usage'].fillna(method='ffill')


In [5]:
from autogluon.tabular import TabularPredictor
autogluon_automl = TabularPredictor(label='electricity_usage', problem_type='regression', eval_metric='mean_squared_error')
autogluon_automl.fit(train_data=df_train, time_limit=5*60)

ModuleNotFoundError: No module named 'autogluon'

In [ ]:
!pip install autogluon

In [ ]:
autogluon_automl.leaderboard()

model	score_val	pred_time_val	fit_time	pred_time_val_marginal	fit_time_marginal	stack_level	can_infer	fit_order
0	WeightedEnsemble_L2	-0.348620	0.303443	287.461536	0.002577	0.276444	2	True	12
1	LightGBMXT	-0.351087	0.010267	1.051211	0.010267	1.051211	1	True	3
2	LightGBM	-0.351415	0.006418	0.353956	0.006418	0.353956	1	True	4
3	CatBoost	-0.353631	0.005425	4.793479	0.005425	4.793479	1	True	6
4	XGBoost	-0.353716	0.009078	1.165040	0.009078	1.165040	1	True	9

In [ ]:
autogluon_automl.evaluate(df_test)

In [ ]:

y_pred = autogluon_automl.predict(df_test)
autogluon_compare = pd.DataFrame(data={'actual': df_test['electricity_usage'], 'predicted': y_pred})
autogluon_compare.plot(figsize=(10, 5))